In [5]:
import tensorflow as tf
import numpy as np
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, ZeroPadding2D, Add, Input, LeakyReLU, UpSampling2D, Concatenate, Lambda, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.losses import binary_crossentropy, sparse_categorical_crossentropy


In [6]:
_ANCHORS = np.array([(10, 13), (16, 30), (33, 23), (30, 61), (62, 45),
                         (59, 119), (116, 90), (156, 198), (373, 326)], np.float32) / 416

In [7]:
_ANCHORS

array([[0.02403846, 0.03125   ],
       [0.03846154, 0.07211538],
       [0.07932692, 0.05528846],
       [0.07211538, 0.14663461],
       [0.14903846, 0.10817308],
       [0.14182693, 0.28605768],
       [0.27884614, 0.21634616],
       [0.375     , 0.47596154],
       [0.89663464, 0.78365386]], dtype=float32)